**<h1><center>PROJECT DATA ANALYSIS</center></h1>**

Thành viên:

|STT|MSSV|Họ tên|
|---|---|---|
|1|||
|2|||
|3|||

# 0. Chọn nguồn dữ liệu

Link: https://catalog.data.gov/dataset/local-weather-archive

# 1. Thuyết minh dữ liệu

Bộ dữ liệu này chứa dữ liệu thời tiết sân bay quốc tế `Raleigh Durham` được lấy từ dịch vụ web `NOAA` được mô tả tại dữ liệu khí hậu trực tuyến: Tài liệu dịch vụ web. 

Chúng tôi đã rút dữ liệu này và chuyển đổi nó thành các đơn vị thường được sử dụng. 

Bộ dữ liệu này là một kho lưu trữ - nó không được cập nhật.

Thông tin thuộc tính

|Thuộc tính|Mô tả|Đơn vị|
|---|---|---|
|date|Thời gian|năm-tháng-ngày|
|tmin|Nhiệt độ thấp nhất|$^oF$|
|tmax|Nhiệt độ cao nhất|$^oF$|
|prcp|Lượng mưa|inch|
|snow|Lượng tuyết rơi|inch|
|snwd|Độ sâu tuyết|inch|
|awnd|Tốc độ gió trung bình|m/h|

# 2. Đặt câu hỏi tình huống, vấn đề tìm hiểu trên bộ dữ liệu

1. Nhiệt độ cao nhất/thấp nhất của mỗi năm là bao nhiêu?
2. Mưa nhiều thường tập trung khoảng thời gian (tháng) nào trong năm?
3. Tuyết thường rơi vào tháng nào, nhiều hay không?
4. Gió có nhiều hay không?

...


# 3. Thực hiện các vấn đề

In [1]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = 'plotly_dark'

Đọc dữ liệu

In [2]:
df = pd.read_csv('weather-history.csv', parse_dates =["date"], index_col ="date") # , index_col ="date"
df.head()

,tmin,tmax,prcp,snow,snwd,awnd
date,,,,,,
2017-01-09,9,31,0.00,0.0,1.2,2.46
2017-01-11,40,57,0.00,0.0,0.0,6.04
2017-01-19,36,63,0.00,0.0,0.0,1.34
2017-01-20,46,59,0.09,0.0,0.0,2.91
2017-01-24,41,63,0.00,0.0,0.0,6.04


In [3]:
ele = {'tmin': 'Nhiệt độ thấp nhất', 'tmax': 'Nhiệt độ cao nhất', 'prcp': 'Lượng mưa', 'snwd': 'Lượng tuyết rơi', 'awnd': 'Tốc độ gió trung bình'}

Xem số lượng dòng, cột của tập dữ liệu

In [4]:
df.shape

(1977, 6)

Xem kiểu dữ liệu các trường dữ liệu

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1977 entries, 2017-01-09 to 2022-05-22
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tmin    1977 non-null   int64  
 1   tmax    1977 non-null   int64  
 2   prcp    1977 non-null   float64
 3   snow    1977 non-null   float64
 4   snwd    1976 non-null   float64
 5   awnd    1973 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 108.1 KB


Kiểm tra dữ liệu `NaN`

In [6]:
df.isna().sum()

tmin    0
tmax    0
prcp    0
snow    0
snwd    1
awnd    4
dtype: int64

Điền dữ liệu `NaN` bằng những giá trị phía dưới (tức ngày hôm sau)

In [7]:
df.fillna(method='bfill', inplace=True)
df.isna().sum()

tmin    0
tmax    0
prcp    0
snow    0
snwd    0
awnd    0
dtype: int64

Xem thống kê tổng quan

In [8]:
df.describe()

,tmin,tmax,prcp,snow,snwd,awnd
count,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000,1977.000000
mean,51.088012,72.298938,0.135771,0.012838,0.016945,6.075488
std,15.889527,15.796087,0.370521,0.227433,0.215284,2.952269
min,4.000000,27.000000,0.000000,0.000000,0.000000,0.000000
25%,37.000000,60.000000,0.000000,0.000000,0.000000,3.800000
50%,52.000000,74.000000,0.000000,0.000000,0.000000,5.800000
75%,66.000000,86.000000,0.060000,0.000000,0.000000,7.800000
max,78.000000,102.000000,4.960000,7.000000,5.900000,20.360000


## Trực quan

### Tổng quan

In [9]:
val = 'sum'
yearly = None
match val:
    case 'sum':
        yearly = df.resample('Y').sum()
    case 'min': 
        yearly = df.resample('Y').min()
    case 'max':
        yearly = df.resample('Y').max()
    case 'mean': 
        yearly = df.resample('Y').mean()
    case 'median': 
        yearly = df.resample('Y').median()

Nhiệt độ

In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=yearly.index.year, y=yearly['tmin'],
    name=ele['tmin'],
    connectgaps=True,
    showlegend = False)
)
fig.add_trace(go.Scatter(x=yearly.index.year, y=yearly['tmax'],
    name=ele['tmax'],
    connectgaps=True,
    showlegend = False)
)
fig.update_layout(
                title={
                    'text': f'<b>Tổng nhiệt độ của từng năm</b>',
                    'y':0.8,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                title_font_color="red",
                font_color="yellow",
                hovermode="x unified",
                hoverlabel=dict(
                    bgcolor="gray",
                    font_size=16,
                    font_family="Rockwell")
)
fig.show()

Lượng mưa

In [11]:
fig = go.Figure()
fig.add_trace(go.Bar(y = yearly.index.year, x=yearly.prcp, text = yearly.prcp, textposition='auto', orientation='h'))
fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,
                  )
fig.update_layout(
    title={
        # 'text': f'<b>Tổng lượng mưa của từng năm</b>',
        # 'y':0.8,
        # 'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font_color="red",
    font_color="yellow",
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="gray",
        font_size=16,
        font_family="Rockwell")
)
fig.update_layout(yaxis_title=None, xaxis_title=None, margin=dict(l=0, r=0, t=0, b=0),
    height=300, width=960)
fig.show()

> Chọn biểu đồ lượng mưa

In [12]:
1920/2

960.0

Lượng tuyết rơi

In [13]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=yearly.index.year, values=yearly.snow, name="Lượng tuyết rơi"), 1, 1)
fig.add_trace(go.Pie(labels=yearly.index.year, values=yearly.snwd, name="Độ dày tuyết"), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo=f"label+value+name")

fig.update_layout(
    title_text="Tổng lượng tuyết rơi và độ dày tuyết hàng năm",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='<b>SNOW</b>', x=0.2, y=0.5, font_size=20, showarrow=False),
                 dict(text='<b>DEPTH</b>', x=0.8, y=0.5, font_size=20, showarrow=False)])
fig.show()

Tốc độ gió trung bình

In [14]:
fig = go.Figure(data=go.Scatterpolar(
  r=yearly.awnd,
  theta=list(map(lambda x: str(x), yearly.index.year.values)),
  fill='toself'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False,
  title = 'Tốc độ gió trung bình hàng năm'
)

fig.show()

In [15]:
fig = go.Figure(go.Barpolar(
    r=yearly.awnd,
    theta=list(map(lambda x: str(x), yearly.index.year.values)),
    width=[0.8, 0.6, 0.4, 0.5, 0.3, 0.2],
    marker_color=["#4618DF", '#709BFF', '#2AC93A', '#FFAA70', '#D50909', '#FFDF70'],
    marker_line_color="yellow",
    marker_line_width=2,
    opacity=0.8
))

fig.update_layout(
    
)
fig.show()

In [16]:
fig = go.Figure()
fig.add_trace(go.Histogram(x = df['tmin'], name = ele['tmin']))
fig.add_trace(go.Histogram(x = df['tmax'], name = ele['tmax']))
# fig.update_layout(barmode='overlay', title_text=f'Nhiệt độ')
fig.update_traces(opacity=0.75)
fig.update_layout(
    barmode='overlay',
    title={
        'text': f'<b>Tổng nhiệt độ</b>',
        'y':0.8,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font_color="red",
    font_color="yellow",
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="gray",
        font_size=16,
        font_family="Rockwell"),
    showlegend = False
)
fig.show()

> Chọn biểu đồ nhiệt độ

Biểu đồ tuyết rơi

In [17]:
fig = go.Figure()
m = df.resample('M').sum()
fig.add_trace(go.Scatter(x=m.index, y=m.snow, fill='tozeroy', mode='none',
                        name = 'Lượng tuyết rơi'))
fig.add_trace(go.Scatter(x=m.index, y=m.snwd, fill='tozeroy', mode='none',
                        name = 'Độ dày tuyết'))

fig.update_layout(
    barmode='overlay',
    title={
        'text': f'<b>Lượng tuyết rơi và độ dày</b>',
        'y':0.8,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font_color="red",
    font_color="yellow",
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="gray",
        font_size=16,
        font_family="Rockwell"),
    showlegend = False
)
fig.update_layout(yaxis_title=None, xaxis_title=None, margin=dict(l=0, r=0, t=0, b=0),
    height=300, width=1960)
fig.show()

> Chọn biểu đồ tuyết 

# Theo từng năm

In [54]:
y = 2020
year = df[df.index.year == y]
monthly = year.resample('M').sum()
x_m = monthly.tmax.max()
y_m = monthly.loc[monthly.tmax == x_m].index.month[0]
x_m, y_m

(2887, 7)

In [59]:
y = 2020
year = df[df.index.year == y]
monthly = year.resample('M').sum()
y_m = monthly.tmax.max()
x_m = monthly.loc[monthly.tmax == y_m].index.month[0]
fig = go.Figure()
fig.add_trace(go.Scatter(x=monthly.index.month, y=monthly['tmin'],
    name=ele['tmin'],
    connectgaps=True)
)

fig.add_trace(go.Scatter(x=monthly.index.month, y=monthly['tmax'],
    name=ele['tmax'],
    connectgaps=True,)
)
fig.update_layout(
    barmode='overlay',
    # title={
    #     'text': f'<b>Tổng nhiệt độ của năm {y}</b>',
    #     'y':0.8,
    #     'x':0.5,
    #     'xanchor': 'center',
    #     'yanchor': 'top'},
    title_font_color="red",
    font_color="yellow",
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="gray",
        font_size=16,
        font_family="Rockwell"),
    showlegend = False
)
fig.add_annotation(
        x=x_m,
        y=y_m,
        xref="x",
        yref="y",
        text="max=2816",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#00e600",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )
fig.update_layout(yaxis_title=None, xaxis_title=None, margin=dict(l=5, r=5, t=5, b=5))
fig.show()

In [64]:
monthly = year.resample('M').sum()
fig = px.bar(x = monthly.index.month, y=monthly['prcp'],
            text_auto=True,
            labels={
                "x": "",
                "y": ""
                })
fig.update_layout(
    barmode='overlay',
    title_font_color="red",
    font_color="yellow",
    showlegend = False,
    plot_bgcolor='black'
)
fig.update_layout(yaxis_title=None, xaxis_title=None, margin=dict(l=5, r=5, t=5, b=5))
fig.show()

In [88]:
fig = px.line_polar(monthly, r = 'awnd', theta=list(map(lambda x: str(x), monthly.index.month.values)),
            labels={
                "awnd": "Tốc độ gió",
                "theta": "Tháng"},
            line_close=True,
            
            )
fig.update_layout(
    barmode='overlay',
    font_color="yellow",
    showlegend = False
)
fig.update_traces(fill='toself')
fig.update_traces(mode="markers+lines")
fig.show()

C:\Users\quach\AppData\Roaming\Python\Python310\site-packages\plotly\express\_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [89]:
fig = go.Figure(data=go.Scatterpolar(
  r=monthly.awnd,
  theta=list(map(lambda x: str(x), monthly.index.month.values)),
  fill='toself',
  name = 'Tốc độ gió trung bình'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False,
  
)
fig.update_layout(
    barmode='overlay',
    font_color="yellow",
    hovermode="x unified",
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="Rockwell"),
    showlegend = False
)
fig.update_layout(yaxis_title=None, xaxis_title=None)
fig.update_layout(yaxis_title=None, xaxis_title=None, margin=dict(l=5, r=5, t=5, b=5))
fig.show()

In [26]:
y = df[df.index.year == 2021]

In [33]:
monthly

,tmin,tmax,prcp,snow,snwd,awnd
date,,,,,,
2021-01-31,986,1553,5.42,1.6,2.0,129.30
2021-02-28,931,1443,6.17,0.0,0.0,180.75
2021-03-31,1345,2045,2.43,0.0,0.0,201.57
2021-04-30,1404,2212,0.94,0.0,0.0,186.57
2021-05-31,1673,2466,1.65,0.0,0.0,173.37
2021-06-30,1992,2571,7.57,0.0,0.0,186.77
2021-07-31,2158,2751,5.03,0.0,0.0,197.75
2021-08-31,2191,2816,3.27,0.0,0.0,149.20
2021-09-30,1853,2524,2.86,0.0,0.0,156.15


In [75]:
fig = go.Figure()
fig.add_trace(go.Box(
    y=monthly.tmax,
    name='',
    jitter=0.3,
    pointpos=-1.8,
    boxpoints='all',
    marker_color='#66ff33',
    line_color='#ff66cc'
))
fig.update_layout(yaxis_title=None, xaxis_title=None, margin=dict(l=5, r=5, t=5, b=5))
fig.show()

In [76]:
fig = go.Figure()
fig.add_trace(go.Box(
    y=monthly.prcp,
    name='',
    jitter=0.3,
    pointpos=-1.8,
    boxpoints='all',
    marker_color='#66ff33',
    line_color='#ff66cc'
))
fig.update_layout(yaxis_title=None, xaxis_title=None, margin=dict(l=5, r=5, t=5, b=5))
fig.show()

### theo ngày

In [23]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = 450,
    mode = "gauge+number+delta",
    title = {'text': "Speed"},
    delta = {'reference': 380},
    gauge = {'axis': {'range': [None, 500]},
             'steps' : [
                 {'range': [0, 250], 'color': "lightgray"},
                 {'range': [250, 400], 'color': "gray"}],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 490}}))

fig.show()

# 4. Thực hiện thông qua một hoặc hai dashboard